In [50]:
import pandas as pd
import math
import json

In [21]:
df = pd.read_csv("attackmatrix_winevent.csv")
df.loc[0].values.tolist()

In [52]:
ttlist = []

for i in range(len(df)):
    rowcal = df.loc[i].values.tolist()
    
    eventlist = []
    for rowitem in rowcal[3:]:
        
        if not pd.isnull(rowitem):
            eventlist.append(int(rowitem))
    
    ttlist.append({
      "EventID" :  eventlist,
        "Tactic" : rowcal[0],
        "TechniqueName" : rowcal[1],
        "TechniqueID" : rowcal[2]
    })
ttlist

[{'EventID': [4688, 4663],
  'Tactic': 'Collection',
  'TechniqueName': 'Audio Capture',
  'TechniqueID': 'T1123'},
 {'EventID': [4688, 4663],
  'Tactic': 'Collection',
  'TechniqueName': 'Automated Collection',
  'TechniqueID': 'T1119'},
 {'EventID': [4688, 4688, 200, 4663, 5861, 500, 4100, 4104],
  'Tactic': 'Collection',
  'TechniqueName': 'Data from Local System',
  'TechniqueID': 'T1005'},
 {'EventID': [4688, 4688, 5140, 4663, 5145],
  'Tactic': 'Collection',
  'TechniqueName': 'Data from Network Shared Drive',
  'TechniqueID': 'T1039'},
 {'EventID': [4688, 4688, 4657, 4663, 5140, 5145],
  'Tactic': 'Collection',
  'TechniqueName': 'Data from Removable Media',
  'TechniqueID': 'T1025'},
 {'EventID': [4688, 4688, 4663],
  'Tactic': 'Collection',
  'TechniqueName': 'Data Staged',
  'TechniqueID': 'T1074'},
 {'EventID': [4688, 5156, 4624, 4663],
  'Tactic': 'Collection',
  'TechniqueName': 'Email Collection',
  'TechniqueID': 'T1114'},
 {'EventID': [4624, 4688],
  'Tactic': 'Collecti

In [53]:
outputfile_json = ".\\mitre_winevent_lookup.json"
with open(outputfile_json, 'w') as filewriter:
    json.dump(ttlist,filewriter)

# Link and populate to neo4j graph

In [63]:
from py2neo import Graph, Node, Relationship, NodeMatcher, cypher

graph = Graph(password="admin")
nodes = NodeMatcher(graph)

In [66]:
for item in ttlist:
    existing_technique_node1 = nodes.match('Technique',TechniqueId=item['TechniqueID']).first()

    for evtx in item['EventID']:
        evtx_node2 = Node("Event", name = evtx)
        graph.create(evtx_node2)
        
        existing_u1_knows_u2 = Relationship(existing_technique_node1, 'has', evtx_node2)
        graph.create(existing_u1_knows_u2)
